In [1]:
import cv2
import numpy as np

In [2]:
def detect_board(vid_frame_gray, ref_frame_gray, orb, bf):
    kp_board, des_board = orb.detectAndCompute(ref_frame_gray, None)
    kp_frame, des_frame = orb.detectAndCompute(vid_frame_gray, None)

    matches = bf.knnMatch(des_board, des_frame, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) > 20:
        src_pts = np.float32([kp_board[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    h, w = ref_frame_gray.shape[:2]


    H_inv = np.linalg.inv(H)
    board_corners = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
    camera_corners = cv2.perspectiveTransform(board_corners, H_inv)
    return camera_corners

In [3]:
board = cv2.imread("../data/board_reference.jpg")
board_gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)

cap = cv2.VideoCapture("../data/easy2_mid.mp4")
ret, frame = cap.read()

frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [4]:
orb = cv2.ORB_create(5000)
bf = cv2.BFMatcher(cv2.NORM_HAMMING)

In [5]:
cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracking", 900, 600)


i = 0
try:
    while True:
        ret, frame = cap.read()
        overlay = frame.copy()
        if i % 10 == 0:
            camera_corners = detect_board(cv2.cvtColor(
                frame, cv2.COLOR_BGR2GRAY), board_gray, orb, bf)

        if camera_corners is not None:
            pts = camera_corners.astype(int)
            cv2.polylines(overlay, [pts], isClosed=True,
                          color=(0, 255, 0), thickness=3)

        i = (i+1) % 1000
        cv2.imshow("Tracking", overlay)

        if cv2.waitKey(30) & 0xFF == 27:
            break
except Exception as e:
    print(e)
finally:
    cap.release()
    cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cezary/PUT_sem5/CV/board_game_tracker/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
